In [1]:
import glob
import math
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.models.vision_transformer import ViT_B_16_Weights
from torchvision.models import EfficientNet_B7_Weights

In [2]:
# settings
input_dir = "./image"
output_train_dir = "./train"
output_test_dir = "./test"
path_list = [f"{output_train_dir}/00_notlike/*",
             f"{output_train_dir}/01_like/*",
             f"{output_test_dir}/*"]

classes = ["00_notlike", "01_like"]

batch_size = 32

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# model loading
net = models.efficientnet_b7(weights=EfficientNet_B7_Weights.DEFAULT)
for param in net.parameters():
    param.requires_grad = False
net.classifier[1] = nn.Linear(2560, 2)
_ = net.cuda()  # GPU対応
# net = models.vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
# for param in net.parameters():
#     param.requires_grad = False
# net.heads[0] = nn.Linear(768, 2)
# _ = net.cuda()  # GPU対応
# saved model
model_path = 'model.pth'
net.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [3]:
ids = pd.DataFrame()
for path in path_list:
    ids = pd.concat([
        ids,
        pd.DataFrame({
            "id": map(lambda x: int(x.split("/")[-1].split(".")[0]),
                      glob.glob(path)),
            "dir": path
        })
    ]).drop_duplicates().reset_index(drop=True)
ids["png"] = ids["dir"].apply(lambda x: x[:-1]) + ids["id"].astype(str) + ".png"
ids["image"] = input_dir + "/" + ids["id"].astype(str) + ".png"

In [4]:
interval = batch_size
ilim = math.ceil(len(ids)/interval)

net.eval()
results = pd.DataFrame()
for i in range(ilim):
    imin = interval*i
    imax = interval*(i+1)

    imgs = []
    index = []
    for index_el, path in zip(ids["id"][imin:imax], ids["png"][imin:imax]):
        img = transform_valid(Image.open(path))
        imgs.append(img)
        index.append(index_el)

    result = pd.DataFrame(
        (net(torch.stack(imgs).cuda())).to('cpu').detach().numpy().copy(),
        columns=classes,
        index=index
    )
    results = pd.concat([results, result])

In [5]:
results.to_csv("prediction.csv", index_label="id")